In [1]:
import scanpy as sc
from transVAE.model.transVAE import transVAE
from transVAE.module._utils import check_adatas_var_index, compute_r2_score

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


In [4]:
# Fixed configurations
max_epochs = 1
dataset_to_predict = "wang_to_predict_3000hvg.h5ad"
dataset_ground_truth1 = "dbdb_ground_truth_3000hvg.h5ad"
dataset_ground_truth2 = "mSTZ_ground_truth_3000hvg.h5ad"
cov_embed_dims = 10
kl_weight = 0.005
save_folder_name = "save_test"
save_models = True
learning_rate = 0.0001
n_layers_fixed = 4
n_latent = 512
n_hidden = 1000

# Grid configurations
# Directly setting values from the options
addl_dataset_train = "embedding_top30_mouse_3000hvg.h5ad"
addl_dataset_train = "None"
n_layers = 4
dropout_rate = 0.2
weight_decay = 0.0001
seed = 42

In [3]:
adata_baseline = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/train_adata_baseline_3000hvg.h5ad")

if addl_dataset_train == "None":
    adata_train = adata_baseline
else:
    adata_extra = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/{addl_dataset_train}")
    adata_train = sc.concat([adata_baseline, adata_extra], join = "inner", axis = 0)

adata_train = transVAE.setup_anndata(adata_train, categorical_covariate_embed_keys=["dataset"], categorical_covariate_keys=["organism"], copy = True)
adata_to_predict = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/data_to_predict/{dataset_to_predict}")
dbdb_ground_truth = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/ground_truth/{dataset_ground_truth1}")
mSTZ_ground_truth = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/ground_truth/{dataset_ground_truth2}")

INFO     Using column names from columns of adata.obsm['covariates']                                               
INFO     Using column names from columns of adata.obsm['covariates_embed']                                         


In [66]:
adata_train

AnnData object with n_obs × n_vars = 204754 × 3000
    obs: 'disease', 'dataset', 'organism', 'cell_type'
    uns: 'covariate_orders', 'covariates_dict', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: 'covariates', 'covariates_embed'

In [12]:
adata_baseline

AnnData object with n_obs × n_vars = 204754 × 3000
    obs: 'disease', 'dataset', 'organism', 'cell_type'

In [13]:
adata_extra

AnnData object with n_obs × n_vars = 96312 × 3000
    obs: 'dataset', 'cell_type', 'disease', 'organism', 'n_counts'
    uns: 'log1p'

In [14]:
adata_train

AnnData object with n_obs × n_vars = 301066 × 3000
    obs: 'disease', 'dataset', 'organism', 'cell_type'
    uns: 'covariate_orders', 'covariates_dict', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: 'covariates', 'covariates_embed'

In [5]:
# make the model
model = transVAE(adata_train, 
                 n_hidden=n_hidden, 
                 n_latent=n_latent, 
                 n_layers=n_layers, 
                 dropout_rate=dropout_rate, 
                 cov_embed_dims = cov_embed_dims, 
                 kl_weight = kl_weight)

# train the model
model.train(batch_size=4096, max_epochs = max_epochs, train_size = 0.9, enable_progress_bar = True, 
            early_stopping = True, early_stopping_monitor = 'reconstruction_loss_validation', early_stopping_min_delta = 0.1, early_stopping_patience = 100,
            plan_kwargs = {"lr": learning_rate, 
                           "weight_decay":weight_decay,
                           "reduce_lr_on_plateau":True,
                           "lr_factor":0.5,
                           "lr_patience":50,
                           "lr_scheduler_metric":"reconstruction_loss_validation"})

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
SLURM auto-requeueing enabled. Setting signal handlers.


Epoch 1/1: 100%|██████████| 1/1 [00:04<00:00,  4.92s/it, v_num=1, train_loss_step=221, train_loss_epoch=317]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 1/1: 100%|██████████| 1/1 [00:04<00:00,  4.94s/it, v_num=1, train_loss_step=221, train_loss_epoch=317]


In [6]:
    adata_to_predict_h = adata_to_predict[adata_to_predict.obs.disease == "healthy"]
    adata_to_predict_OOD = adata_to_predict[adata_to_predict.obs.disease == "T2D"]

    preds_dbdb_h = model.translate(adata_to_predict_h, translate_dict={"dataset":"db/db", "organism": "Mus musculus"})
    preds_dbdb_OOD = model.translate(adata_to_predict_OOD, translate_dict={"dataset":"db/db", "organism": "Mus musculus"})

    dbdb_ground_truth_h = dbdb_ground_truth[dbdb_ground_truth.obs.disease == "healthy"]
    dbdb_ground_truth_OOD = dbdb_ground_truth[dbdb_ground_truth.obs.disease == "T2D"]
    
    preds_mSTZ_h = model.translate(adata_to_predict_h, translate_dict={"dataset":"mSTZ", "organism": "Mus musculus"})
    preds_mSTZ_OOD = model.translate(adata_to_predict_OOD, translate_dict={"dataset":"mSTZ", "organism": "Mus musculus"})

    mSTZ_ground_truth_h = mSTZ_ground_truth[mSTZ_ground_truth.obs.disease == "healthy"]
    mSTZ_ground_truth_OOD = mSTZ_ground_truth[mSTZ_ground_truth.obs.disease == "T2D"]

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


In [9]:
from transVAE.module._utils import compare_de

In [23]:
help(compare_de)

Help on function compare_de in module transVAE.module._utils:

compare_de(X: numpy.ndarray, Y: numpy.ndarray, C: numpy.ndarray, shared_top: int = 100, **kwargs) -> dict
    Compare DEG across real and simulated perturbations.
    
    Computes DEG for real and simulated perturbations vs. control and calculates
    metrics to evaluate similarity of the results.
    
    Args:
        X: Real perturbed data.
        Y: Simulated perturbed data.
        C: Control data
        shared_top: The number of top DEG to compute the proportion of their intersection.
        **kwargs: arguments for `scanpy.tl.rank_genes_groups`.



In [10]:
    results = model.history

    results["r2_dbdb_healthy"] = compute_r2_score(preds_dbdb_h, dbdb_ground_truth_h)
    results["r2_dbdb_OOD"] = compute_r2_score(preds_dbdb_OOD, dbdb_ground_truth_OOD)
    results["r2_mSTZ_healthy"] = compute_r2_score(preds_mSTZ_h, mSTZ_ground_truth_h)
    results["r2_mSTZ_OOD"] = compute_r2_score(preds_mSTZ_OOD, mSTZ_ground_truth_OOD)

In [11]:
results_de_dbdb = compare_de(X=dbdb_ground_truth_OOD.X.todense(), Y=preds_dbdb_OOD.X, C=dbdb_ground_truth_h.X.todense())

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:420: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(


In [13]:
results_de_mSTZ = compare_de(X=mSTZ_ground_truth_OOD.X.todense(), Y=preds_mSTZ_OOD.X, C=mSTZ_ground_truth_h.X.todense())

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:420: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(


In [14]:
for name, value in results_de_dbdb.items():
    results[name + "_dbdb"] = value

In [15]:
for name, value in results_de_mSTZ.items():
    results[name + "_mSTZ"] = value

In [17]:
results

{'kl_weight':       kl_weight
 epoch          
 0           0.0,
 'train_loss_step':       train_loss_step
 epoch                
 0          232.503235,
 'validation_loss':       validation_loss
 epoch                
 0          250.413864,
 'elbo_validation':       elbo_validation
 epoch                
 0          501.629791,
 'reconstruction_loss_validation':       reconstruction_loss_validation
 epoch                               
 0                           500.8237,
 'kl_local_validation':       kl_local_validation
 epoch                    
 0                  0.8061,
 'kl_global_validation':       kl_global_validation
 epoch                     
 0                      0.0,
 'train_loss_epoch':       train_loss_epoch
 epoch                 
 0           316.614014,
 'elbo_train':         elbo_train
 epoch             
 0      2086.447754,
 'reconstruction_loss_train':       reconstruction_loss_train
 epoch                          
 0                    625.925354,
 'kl_loc

In [22]:
X=mSTZ_ground_truth_OOD.X.todense()
Y=preds_mSTZ_OOD.X
C=mSTZ_ground_truth_h.X.todense()
n_vars = X.shape[1]
shared_top =100

import numpy as np
import anndata as ad
import pandas as pd

assert n_vars == Y.shape[1] == C.shape[1]

shared_top = min(shared_top, n_vars)
vars_ranks = np.arange(1, n_vars + 1)

adatas_xy = {}
adatas_xy["x"] = ad.AnnData(X, obs={"label": "comp"})
adatas_xy["y"] = ad.AnnData(Y, obs={"label": "comp"})
adata_c = ad.AnnData(C, obs={"label": "ctrl"})

results = pd.DataFrame(index=adata_c.var_names)
top_names = []
for group in ("x", "y"):
    adata_joint = ad.concat((adatas_xy[group], adata_c), index_unique="-")

    sc.tl.rank_genes_groups(adata_joint, groupby="label", reference="ctrl", key_added="de")

    srt_idx = np.argsort(adata_joint.uns["de"]["names"]["comp"])
    results[f"scores_{group}"] = adata_joint.uns["de"]["scores"]["comp"][srt_idx]
    results[f"pvals_adj_{group}"] = adata_joint.uns["de"]["pvals_adj"]["comp"][srt_idx]
    # needed to avoid checking rankby_abs
    results[f"ranks_{group}"] = vars_ranks[srt_idx]

    top_names.append(adata_joint.uns["de"]["names"]["comp"][:shared_top])

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:420: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(


In [45]:
import os
import numpy as np

In [51]:
if save_models:
    save_dir = f"/d/hpc/projects/FRI/DL/mo6643/MSC/cross_species_prediction_save/{save_folder_name}"
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_dir = os.path.join(save_dir, f"transVAE_train_{addl_dataset_train}_hid{n_hidden}_lat{n_latent}_lr{learning_rate}_cov{cov_embed_dims}_ep{max_epochs}_ly{n_layers}_dr{dropout_rate}_kl{kl_weight}_wd{weight_decay}_s{seed}")
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    np.save(arr=preds_dbdb_h.X, file=os.path.join(save_dir,"preds_dbdb_h.npy"))
    np.save(arr=preds_dbdb_OOD.X, file=os.path.join(save_dir,"preds_dbdb_OOD.npy"))
    np.save(arr=preds_mSTZ_h.X, file=os.path.join(save_dir,"preds_mSTZ_h.npy"))
    np.save(arr=preds_mSTZ_OOD.X, file=os.path.join(save_dir,"preds_mSTZ_OOD.npy"))

In [52]:
save_dir

'/d/hpc/projects/FRI/DL/mo6643/MSC/cross_species_prediction_save/save_test/transVAE_train_embedding_top30_mouse_3000hvg.h5ad_hid1000_lat512_lr0.0001_cov10_ep2000_ly4_dr0.2_kl0.005_wd0.0001_s42'

In [56]:
    import logging
    logging.info(f'Received the following configuration for datasets train: {addl_dataset_train}, shape: {adata_train.shape}, baseline_shape: {adata_baseline.shape}')
    logging.info(f'n_hidden: {n_hidden}, n_latent: {n_latent}, learning_rate: {learning_rate}, cov_embed_dims: {cov_embed_dims}, weight_decay {weight_decay}')
    logging.info(f'max_epochs: {max_epochs}, n_layers: {n_layers}, dropout_rate: {dropout_rate}, kl_weight: {kl_weight}')

In [60]:
adata_train.shape

(301066, 3000)

In [61]:
adata_baseline.shape

(204754, 3000)

In [36]:
Y[Y < 0] = 0

In [38]:
np.mean(Y = 0)

0.4185192063750203

In [ ]:
def compare_logfold(X: np.ndarray, Y: np.ndarray, C: np.ndarray, **kwargs) -> dict:
    """Compare DEG across real and simulated perturbations.

    Computes DEG for real and simulated perturbations vs. control and calculates
    metrics to evaluate similarity of the results.

    Args:
        X: Real perturbed data.
        Y: Simulated perturbed data.
        C: Control data
        shared_top: The number of top DEG to compute the proportion of their intersection.
        **kwargs: arguments for `scanpy.tl.rank_genes_groups`.
    """
    n_vars = X.shape[1]
    assert n_vars == Y.shape[1] == C.shape[1]

    prop_of_genes_set_to_0 = np.mean(Y < 0)
    Y[Y < 0] = 0

    adatas_xy = {}
    adatas_xy["x"] = ad.AnnData(X, obs={"label": "comp"})
    adatas_xy["y"] = ad.AnnData(Y, obs={"label": "comp"})
    adata_c = ad.AnnData(C, obs={"label": "ctrl"})

    results = pd.DataFrame(index=adata_c.var_names)
    top_names = []
    for group in ("x", "y"):
        adata_joint = ad.concat((adatas_xy[group], adata_c), index_unique="-")

        sc.tl.rank_genes_groups(adata_joint, groupby="label", reference="ctrl", key_added="de", **kwargs)
        results[f"logfold_{group}"] = [elm[0] for elm in adata_joint.uns["de"]["logfoldchanges"].tolist()]

    metics = {}
    metics["logfold_corr"] = results["logfold_x"].corr(results["logfold_y"], method="pearson")
    metics["prop_of_genes_set_to_0"] = prop_of_genes_set_to_0

    return metics

In [44]:
adata_joint.uns["de"]["logfoldchanges"]

rec.array([(25.385717,), (25.294678,), (25.585024,), ..., (      nan,),
           (      nan,), (      nan,)],
          dtype=[('comp', '<f4')])

In [71]:
X=mSTZ_ground_truth_OOD.X.todense()
Y=preds_mSTZ_OOD.X
C=mSTZ_ground_truth_h.X.todense()
n_vars = X.shape[1]
shared_top =100

n_vars = X.shape[1]
assert n_vars == Y.shape[1] == C.shape[1]

prop_of_genes_set_to_0 = np.mean(Y < 0)
Y[Y < 0] = 0

adatas_xy = {}
adatas_xy["x"] = ad.AnnData(X, obs={"label": "comp"})
adatas_xy["y"] = ad.AnnData(Y, obs={"label": "comp"})
adata_c = ad.AnnData(C, obs={"label": "ctrl"})

results = pd.DataFrame(index=adata_c.var_names)
top_names = []
for group in ("x", "y"):
    adata_joint = ad.concat((adatas_xy[group], adata_c), index_unique="-")

    sc.tl.rank_genes_groups(adata_joint, groupby="label", reference="ctrl", key_added="de")
    results[f"logfold_{group}"] = [elm[0] for elm in adata_joint.uns["de"]["logfoldchanges"].tolist()]

metics = {}
metics["logfold_corr"] = results["logfold_x"].corr(results["logfold_y"], method="pearson")
metics["prop_of_genes_set_to_0"] = prop_of_genes_set_to_0

return metics

SyntaxError: 'return' outside function (873779579.py, line 30)

In [72]:
metics

{'logfold_corr': 0.06261454502009242, 'prop_of_genes_set_to_0': 0.0}

In [50]:
adata_joint.uns["de"]["logfoldchanges"]

rec.array([( 25.385717,), ( 25.294678,), ( 25.585024,), ...,
           ( -8.981566,), (-10.482617,), (-10.139835,)],
          dtype=[('comp', '<f4')])

In [66]:
results = pd.DataFrame(index=adata_c.var_names)
results[f"logfold_{group}"] = [elm[0] for elm in adata_joint.uns["de"]["logfoldchanges"].tolist()]
results

,logfold_y
0,25.385717
1,25.294678
2,25.585024
3,25.271259
4,25.970266
...,...
2995,-7.963964
2996,-8.841400
2997,-8.981566
2998,-10.482617


In [56]:
results = pd.DataFrame(index=adata_c.var_names)


In [75]:
addl_dataset_train == "train_adata_baseline_3000hvg.h5ad"

True

In [74]:
addl_dataset_train = "train_adata_baseline_3000hvg.h5ad"